# Naive Bayes based Spam Filter

Study of probablity teaches us how to 

   - Assign probabilities to events based on certain conditions by using conditional probability rules.
   - Assign probabilities to events based on whether they are in relationship of statistical independence or not with other events.
   - Assign probabilities to events based on prior knowledge by using Bayes' theorem.
   
Using our knowledge of Navie Bayes theorem, we can build numerous practial applications. One such application is spam filter for incoming messages. Our aim is to build a new span filter which categorizes any incoming message as spam/non-spam. Since Naive Bayes relies on calculating probablity of an event based on prior knowledge, we need to build our system with dataset which contains messages that are already categorized as spam/non-spam by humans. 

To classify messages as spam or non-spam based on Naive Bayes algorithm, computer needs

   - Learn how humans classify messages.
   - Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
   - Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

So our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly from [this link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

## Project goal

Our goal is to write a program that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam)


## Dataset exploration

Let's start by reading in the dataset.Our data is stored in a tab separated file `SMSSpamCollection` and has no headers. We will read it and name the columns as `Label` and `SMS`



In [1]:
import pandas as pd

sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

print(sms_spam.shape)
sms_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
sms_spam["Label"].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

The data set contains training data which has nearly 87% non spam SMS and remaining 15% spam SMSes

## Training set and Test set

Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

   - A training set, which we'll use to "train" the computer how to classify messages.
   - A test set, which we'll use to test how good the spam filter is with classifying new messages.

Since we only have one dataset, we will partition into two separate datasets. One will serve as our training set and the second one as our test set. We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

   - The training set will have 4,458 messages (about 80% of the dataset).
   - The test set will have 1,114 messages (about 20% of the dataset).
   
Our test set will have 1114 messages which are already classified by a human. When the spam filter is ready, we're going to treat these messages as new and have the filter classify them. Once we have the results, we'll be able to compare the algorithm classification with that done by a human, and this way we'll see how good the spam filter really is.

In [3]:
def split_test_training_sets(data,training_set_per = 0.8, 
                             test_setper = 0.2):
    """
    this function will partition the dataset provided in first parameter
    into two separate sets, one for training and another for test based on 
    the provided percentages
    
    Args:
        data (pandas DataFrame)- input data set
        training_set_per (float default 0.8 (80%)) = training set percentage
        test_set_per (float default 0.2 (20%)) = test set percentage
    Returns:
        two separate dataframes divided based on training_set_per and test_setper
        in that order
    """
    
    #randomize dataset
    randomized_dataset = data.sample(frac=1, random_state= 1)
    
    training_index = round(len(randomized_dataset) * training_set_per)
    training_set = randomized_dataset[:training_index].reset_index(drop=True)
    test_set = randomized_dataset[training_index:].reset_index(drop=True)
    return training_set, test_set




In [4]:
training_set, test_set = split_test_training_sets(sms_spam)
print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


Lets verify if the training and test sets have same proportion of spam/ham messages as in the original full data set



In [5]:
training_set['Label'].value_counts(normalize=True) * 100


ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [6]:
test_set['Label'].value_counts(normalize=True)* 100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

They are very close to the original dataset. The following table summarizes the percentages of ham/spam in training/test compared to original dataset

|      | Orginal   | Training  | Test      |
|------|-----------|-----------|-----------|
| ham  | 86.593683 | 86.54105  | 86.804309 |
| spam | 13.406317 | 13.45895  | 13.195691 |

# Data cleaning

We have created the datasets for training and testing. The training dataset will be used to train our application in classifying new messages as spam/ham. We will be using Naive Bayes algorithm, which dependens on calculating probability of each word being in spam/ham messages. The probability calculations is based on the word and its occurence count in a given messages. We will need to build  Vocabulary, and build a table which describes the word occurence count for every word in the sms

**FROM**
<img src="https://github.com/renuka28/datascience/blob/master/Building_a_Spam_Filter_with_Naive_Bayes/cpgp_dataset_1.png?raw=true" alt="Original SMS messages" title="Original SMS messages" />

**TO**
<img src="https://github.com/renuka28/datascience/blob/master/Building_a_Spam_Filter_with_Naive_Bayes/cpgp_dataset_2.png?raw=true" alt="new SMS table with word count" title="new SMS table with word count" />

Some notes on the above transformation

   - The SMS column doesn't exist anymore.
   - Instead, the SMS column is replaced by a series of new columns, where each column represents a unique word from the vocabulary.
   - Each row describes a single message. For instance, the first row corresponds to the message "SECRET PRIZE! CLAIM SECRET PRIZE NOW!!", and it has the values **spam, 2, 2, 1, 1, 0, 0, 0, 0, 0.** These values tell us that:
       - The message is spam.
       - The word "secret" occurs two times inside the message.
       - The word "prize" occurs two times inside the message.
       - The word "claim" occurs one time inside the message.
       - The word "now" occurs one time inside the message.
       - The words "coming", "to", "my", "party", and "winner" occur zero times inside the message.
   - All words in the vocabulary are in lower case, so "SECRET" and "secret" come to be considered to be the same word.
   - Punctuation is not taken into account anymore (for instance, we can't look at the table and conclude that the first message initially had three exclamation marks).
   
 
Let's begin the data cleaning process by removing the punctuation and bringing all the words to lower case.

### Step 1 - Make words all lower case

### Step 2 - Remove punctuations



In [7]:
print("Before cleaning")
training_set.head()

Before cleaning


,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [8]:
def clean_training_set(training_set):
    training_set['SMS'] = training_set['SMS'].str.lower()
    training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
    return training_set


In [9]:
training_set = clean_training_set(training_set)
print ("After cleaning")
training_set.head()

After cleaning


,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Create Vocabulary

Vocabulary for the training set is the complete list of all words found in our training set. 


In [10]:
def create_vocabulary(training_set):
    training_set['SMS'] = training_set['SMS'].str.split()
    vocabulary = []
    for sms in training_set['SMS']:
        for word in sms:
            vocabulary.append(word)

    vocabulary = list(set(vocabulary))
    return training_set, vocabulary




In [11]:
training_set, vocabulary = create_vocabulary(training_set)

print("Vocabulary length = {} \n".format(len(vocabulary)))
print("SMS Word list ")
training_set['SMS'].head()

Vocabulary length = 7783 

SMS Word list 


0                    [yep, by, the, pretty, sculpture]
1    [yes, princess, are, you, going, to, make, me,...
2                      [welp, apparently, he, retired]
3                                             [havent]
4    [i, forgot, 2, ask, ü, all, smth, there, s, a,...
Name: SMS, dtype: object

### Final training set

Now that we have our vocabulary, we can go ahead and do our data transformation. 

our target is to covert each SMS to a table of word counts which are stored in a table 

**FROM**
<img src="https://github.com/renuka28/datascience/blob/master/Building_a_Spam_Filter_with_Naive_Bayes/cpgp_dataset_1.png?raw=true" alt="Original SMS messages" title="Original SMS messages" />

**TO**
<img src="https://github.com/renuka28/datascience/blob/master/Building_a_Spam_Filter_with_Naive_Bayes/cpgp_dataset_2.png?raw=true" alt="new SMS table with word count" title="new SMS table with word count" />


the above table will get converted into following dictionary

word_counts_per_sms = {'secret': [2,1,1],
                       'prize': [2,0,1],
                       'claim': [1,0,1],
                       'now': [1,0,1],
                       'coming': [0,1,0],
                       'to': [0,1,0],
                       'my': [0,1,0],
                       'party': [0,1,0],
                       'winner': [0,0,1]
                      }

The table will look like below

| secret | prize | claim | now | coming | to | my | party | winner |   |
|--------|-------|-------|-----|--------|----|----|-------|--------|---|
| 0      | 2     | 2     | 1   | 1      | 0  | 0  | 0     | 0      | 0 |
| 1      | 1     | 0     | 0   | 0      | 1  | 1  | 1     | 1      | 0 |
| 2      | 1     | 1     | 1   | 1      | 0  | 0  | 0     | 0      | 1 |
                   

In [12]:
def create_final_training_set(training_set, vocabulary):
    # lets start by creating an empty dictionary with each word in our vacabulary as
    # key and 0s for each message. After that we will go through all sms and 
    #update the count for each word
    word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}
    for index, sms in enumerate(training_set['SMS']):
        for word in sms:
            word_counts_per_sms[word][index] += 1
    #lets create pandas data frame with the our dictionary
    
    word_counts = pd.DataFrame(word_counts_per_sms)
    
    #lets concatinate orginal sms dataframe with the newly constructed one
    training_set_clean = pd.concat([training_set, word_counts], axis=1)
    return training_set_clean


In [13]:
training_set_clean = create_final_training_set(training_set, vocabulary)

In [14]:
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating Constants First

We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

$$
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
$$$$
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
$$
Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:

$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$
Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

   - P(Spam) and P(Ham)
   - NSpam, NHam, NVocabulary
We'll also use Laplace smoothing and set $\alpha = 1$.

In [15]:
p_spam = "p_spam"
p_ham = "p_ham"
n_spam = "n_spam"
n_ham = "n_ham"
n_vocabulary = "n_vocabulary"
alpha = "alpha"
p_ham_denominator = "p_ham_denominator"
p_spam_denominator = "p_spam_denominator"
spam_messages = "spam_messages"
ham_messages = "ham_messages"

def calculate_constants(training_set_clean):
    constants = {}
    # Isolating spam and ham messages first
    constants[spam_messages] = training_set_clean[training_set_clean['Label'] == 'spam']
    
    constants[ham_messages] = training_set_clean[training_set_clean['Label'] == 'ham']

    # P(Spam) and P(Ham)
    constants[p_spam] = len(constants[spam_messages]) / len(training_set_clean)
    constants[p_ham] = len(constants[ham_messages]) / len(training_set_clean)
    print("p_spam = {} and p_ham = {}".format(constants[p_spam],
                                              constants[p_ham] ))


    # N_Spam
    n_words_per_spam_message = constants[spam_messages]['SMS'].apply(len)
    constants[n_spam] = n_words_per_spam_message.sum()

    # N_Ham
    n_words_per_ham_message = constants[ham_messages]['SMS'].apply(len)
    constants[n_ham] = n_words_per_ham_message.sum()

    print("n_spam = {} and n_ham = {}".format(constants[n_spam],
                                              constants[n_ham] ))

    # N_Vocabulary
    constants[n_vocabulary] = len(vocabulary)
    print("n_vocabulary = {}".format(constants[n_vocabulary]))

    # Laplace smoothing
    constants[alpha] = 1
    
    #calculate denominators which will be constant
    constants[p_ham_denominator] = constants[n_ham] +  (constants[alpha] * constants[n_vocabulary])
    constants[p_spam_denominator] = constants[n_spam] +  (constants[alpha] * constants[n_vocabulary])
    
    return constants


In [16]:
constants = calculate_constants(training_set_clean)
print(constants[spam_messages].shape)
print(constants[ham_messages].shape)

p_spam = 0.13458950201884254 and p_ham = 0.8654104979811574
n_spam = 15190 and n_ham = 57237
n_vocabulary = 7783
(600, 7785)
(3858, 7785)


## Calculating Parameters

we managed to calculate a few terms for our equations:

  - P(Spam) and P(Ham)
  - NSpam, NHam, NVocabulary
  
As we've already mentioned, all these terms will have constant values in our equations for every new message (regardless of the message or each individual word in the message).

However, P(Wi|Spam) and P(Wi|Ham) will vary depending on the individual words. For instance, P("secret"|Spam) will have a certain probability value, while P("cousin"|Spam) or P("lovely"|Spam) will most likely have other values.

Although both P(Wi|Spam) and P(Wi|Ham) vary depending on the word, the probability for each individual word is constant for every new message.

For instance, let's say we receive two new messages:

  - "secret code"
  - "secret party 2night"
We'll need to calculate P("secret"|Spam) for both these messages, and we can use the training set to get the values we need to find a result for the equation below:

The steps we take to calculate P("secret"|Spam) will be identical for both of our new messages above, or for any other new message that contains the word "secret". The key detail here is that calculating P("secret"|Spam) only depends on the training set, and as long as we don't make changes to the training set, P("secret"|Spam) stays constant. The same reasoning also applies to P("secret"|Ham).

This means that we can use our training set to calculate the probability for each word in our vocabulary. If our vocabulary contained only the words "lost", "navigate", and "sea", then we'd need to calculate six probabilities:

  - P("lost"|Spam) and P("lost"|Ham)
  - P("navigate"|Spam) and P("navigate"|Ham)
  - P("sea"|Spam) and P("sea"|Ham)

We have 7,783 words in our vocabulary, which means we'll need to calculate a total of 15,566 probabilities. For each word, we need to calculate both P(Wi|Spam) and P(Wi|Ham).

In more technical language, the probability values that P(wi|Spam) and P(wi|Ham) will take are called **`parameters`**.

Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$


The fact that we calculate so many values before even beginning the classification of new messages makes the Naive Bayes algorithm very fast (especially compared to other algorithms). When a new message comes in, most of the needed computations are already done, which enables the algorithm to almost instantly classify the new message.

In [17]:
def calculate_parameters(constants, vocabulary):
    parameters_spam = {unique_word:0 for unique_word in vocabulary}
    parameters_ham = {unique_word:0 for unique_word in vocabulary}

    # Calculate parameters
    for word in vocabulary:
        parameters_spam[word] = (constants[spam_messages][word].sum() + constants[alpha]) / constants[p_spam_denominator]
        parameters_ham[word] = (constants[ham_messages][word].sum() + constants[alpha]) / constants[p_ham_denominator]
    return parameters_ham, parameters_spam
        

In [18]:
parameters_ham, parameters_spam = calculate_parameters(constants, vocabulary)

In [19]:
len(parameters_ham)

7783

## Classifying A New Message

ow that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

  - Takes in as input a new message (w1, w2, ..., wn)
  - Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
  - Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
      - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
      - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
      - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help

In [20]:
def classify(message, parameters_spam, parameters_ham, constants):
    import re
    '''
    message: a string
    '''
    # remove punctuations 
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = constants[p_spam]
    p_ham_given_message = constants[p_ham]

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
#     print('P(Spam|message):', p_spam_given_message)
#     print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message == p_spam_given_message:
        ret_val = "needs human intervention"
    else:
        ret_val = "ham" if p_ham_given_message > p_spam_given_message else "spam"
    return ret_val


In [21]:
print(classify('WINNER!! This is the secret code to unlock the money: C3421.', parameters_spam, parameters_ham, constants))

spam


In [22]:
print(classify("Sounds good, Tom, then see u there", parameters_spam, parameters_ham, constants))

ham


In [23]:
print(classify("you won the prize", parameters_spam, parameters_ham, constants))

spam


## Measuring the Spam Filter's Accuracy

Hurray!!! our spam filter needs to be working. Lets see how it performs against our test data set of 1114 messages. Note that, in training, our algorithm didn't see these 1,114 messages, so every message in the test set is practically new from the perspective of the algorithm.

In [24]:
def classify_test_set(message):
    return classify(message, parameters_spam, parameters_ham, constants)
    

In [25]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now, we'll write a function to measure the accuracy of our spam filter to find out how well our spam filter does.

We can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:



In [26]:
correct = 0
total = test_set.shape[0]
    
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy: {0:.2f}%'.format(correct/total * 100))

Correct: 1100
Incorrect: 14
Accuracy: 98.74%


The accuracy is close to 98.74%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.


Next Steps
In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set we used, which is a pretty good result. Our initial goal was an accuracy of over 80%, and we managed to do way better than that.

## Next steps include:

   - Analyze the 14 messages that were classified incorrectly and try to figure out why the algorithm classified them incorrectly
   - Make the filtering process more complex by making the algorithm sensitive to letter case

